In [1]:
import datetime
print(datetime.datetime.now())

2021-09-12 09:53:27.044933


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns 

import itertools

import scipy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
df = pd.read_csv("E:\MMA - Smith\Term 2\MMA 823 - Analytics for Financial Markets\Assignment 1\Bankruptcy_data_Final.csv")
df.info()

df = df.dropna()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92872 entries, 0 to 92871
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EPS                 92867 non-null  float64
 1   Liquidity           92625 non-null  float64
 2   Profitability       92625 non-null  float64
 3   Productivity        92625 non-null  float64
 4   Leverage Ratio      92846 non-null  float64
 5   Asset Turnover      92625 non-null  float64
 6   Operational Margin  87315 non-null  float64
 7   Return on Equity    92864 non-null  float64
 8   Market Book Ratio   92815 non-null  float64
 9   Assets Growth       86171 non-null  float64
 10  Sales Growth        86171 non-null  float64
 11  Employee Growth     85862 non-null  float64
 12  BK                  92872 non-null  int64  
dtypes: float64(12), int64(1)
memory usage: 9.2 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 81204 entries, 1 to 92871
Data columns (total 13 column

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth,BK
1,1.41,0.36,0.19,0.12,1.31,1.59,0.07,0.13,2.41,0.126,0.014,0.040,0
2,0.31,0.32,0.13,0.08,1.03,1.55,0.05,0.04,2.56,0.368,0.328,0.567,0
3,0.71,0.28,0.14,0.08,0.80,1.39,0.06,0.05,5.28,-0.021,-0.119,-0.096,0
4,0.75,0.41,0.13,0.08,0.20,1.30,0.06,0.04,8.68,0.233,0.147,0.053,0
5,1.50,0.37,0.16,0.11,0.34,1.41,0.08,0.08,7.82,0.132,0.232,0.077,0


In [5]:
from sklearn.model_selection import train_test_split

# This dataset is huge, so let's take a sample to speed things up
# df = df.sample(frac=0.5, replace=False, random_state=1, axis=0)
X = df.drop(['BK'], axis=1)
y = df['BK']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=44)

In [6]:
y_train.value_counts()
y_test.value_counts()

0    64585
1      378
Name: BK, dtype: int64

0    16146
1       95
Name: BK, dtype: int64

In [7]:
from xgboost import XGBClassifier
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

In [8]:
# Helper function
def quick_evaluate_with_dt(X_train, X_test, y_train, y_test, name, balance_weights=False):
    
    cw = None
    if balance_weights == True:
        cw = 'balanced'
        
    clf = RandomForestClassifier(random_state=0, n_estimators=100, class_weight=cw)
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    accuracy       = accuracy_score(y_test, y_pred)
    f1             = f1_score(y_test, y_pred)
    recall         = recall_score(y_test, y_pred)
    precision      = precision_score(y_test, y_pred)
    roc_auc        = roc_auc_score(y_test, y_pred)
    
    df = pd.DataFrame({"Method"    : [name],
                       "Neg"       : [tn + fn],
                       "True Neg"  : [tn],
                       "False Neg" : [fn],
                       "Pos"       : [tp + fp],
                       "TP"        : [tp],
                       "FP"        : [fp],
                       "Accuracy"  : [accuracy],
                       "Recall"    : [recall],
                       "Precision" : [precision],
                       "F1"        : [f1],
                       "AUC"       : [roc_auc],
                      })
    
    print(df)
    return df

In [9]:
evals = list()

In [10]:
X_train.shape
y_train.shape
np.bincount(y_train)
evals.append(quick_evaluate_with_dt(X_train, X_test, y_train, y_test, 'None'))

(64963, 12)

(64963,)

array([64585,   378], dtype=int64)

  Method    Neg  True Neg  False Neg  Pos  TP  FP  Accuracy  Recall  \
0   None  16240     16145         95    1   0   1  0.994089     0.0   

   Precision   F1       AUC  
0        0.0  0.0  0.499969  


In [11]:
X_train.shape
y_train.shape
np.bincount(y_train)

evals.append(quick_evaluate_with_dt(X_train, X_test, y_train, y_test, 'Class Weights', balance_weights=True))

(64963, 12)

(64963,)

array([64585,   378], dtype=int64)

          Method    Neg  True Neg  False Neg  Pos  TP  FP  Accuracy  Recall  \
0  Class Weights  16241     16146         95    0   0   0  0.994151     0.0   

   Precision   F1  AUC  
0        0.0  0.0  0.5  


C:\Users\irfan\AppData\Local\R-MINI~1\envs\mma\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test, 'Over Random'))

(129170, 12)

(129170,)

array([64585, 64585], dtype=int64)

        Method    Neg  True Neg  False Neg  Pos  TP  FP  Accuracy    Recall  \
0  Over Random  16239     16145         94    2   1   1  0.994151  0.010526   

   Precision        F1       AUC  
0        0.5  0.020619  0.505232  


In [13]:
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test, 'Over SMOTE'))

(129170, 12)

(129170,)

array([64585, 64585], dtype=int64)

       Method    Neg  True Neg  False Neg  Pos  TP   FP  Accuracy    Recall  \
0  Over SMOTE  16007     15939         68  234  27  207  0.983068  0.284211   

   Precision        F1       AUC  
0   0.115385  0.164134  0.635695  


In [14]:
from imblearn.over_sampling import ADASYN

X_resampled, y_resampled = ADASYN(random_state=0).fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test,  'Over ADASYN'))

(129151, 12)

(129151,)

array([64585, 64566], dtype=int64)

        Method    Neg  True Neg  False Neg  Pos  TP   FP  Accuracy    Recall  \
0  Over ADASYN  16015     15949         66  226  29  197  0.983806  0.305263   

   Precision        F1       AUC  
0   0.128319  0.180685  0.646531  


In [15]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test,  'Under Sample'))

(756, 12)

(756,)

array([378, 378], dtype=int64)

         Method    Neg  True Neg  False Neg   Pos  TP    FP  Accuracy  \
0  Under Sample  13719     13704         15  2522  80  2442  0.848716   

     Recall  Precision        F1      AUC  
0  0.842105   0.031721  0.061139  0.84543  


In [16]:
# Recall the actual distrition of the truth labels of the testing set.
y_test.value_counts()

evals_all = pd.concat([m for m in evals], axis = 0).reset_index()

evals_all = evals_all.drop(columns = "index",axis =1)
evals_all.sort_values(by=['F1'], ascending=False)

0    16146
1       95
Name: BK, dtype: int64

,Method,Neg,True Neg,False Neg,Pos,TP,FP,Accuracy,Recall,Precision,F1,AUC
4,Over ADASYN,16015,15949,66,226,29,197,0.983806,0.305263,0.128319,0.180685,0.646531
3,Over SMOTE,16007,15939,68,234,27,207,0.983068,0.284211,0.115385,0.164134,0.635695
5,Under Sample,13719,13704,15,2522,80,2442,0.848716,0.842105,0.031721,0.061139,0.845430
2,Over Random,16239,16145,94,2,1,1,0.994151,0.010526,0.500000,0.020619,0.505232
0,None,16240,16145,95,1,0,1,0.994089,0.000000,0.000000,0.000000,0.499969
1,Class Weights,16241,16146,95,0,0,0,0.994151,0.000000,0.000000,0.000000,0.500000


In [17]:
print(f1_score(y_test,clf.predict(X_test)))
print(accuracy_score(y_test,xgb.predict(X_test)))

NameError: name 'clf' is not defined

In [ ]:
plt.figure(figsize=(16, 10));
plt.grid(True);

ind_d = y==1
ind_p = y==0

plt.scatter(X[ind_p], X[ind_p], marker='o', s=200, label="Did not bankcrupt", alpha=0.8);
plt.scatter(X[ind_d], X[ind_d], marker='o', s=200, label='Bankcrupted');

plt.legend(fontsize=16);
plt.title("Bank Dataset", fontsize=20);
plt.xlabel('Leverage Ratio', fontsize=22);
plt.ylabel('Operational Margin', fontsize=22);
plt.xticks(fontsize=18);
plt.yticks(fontsize=18);
plt.tight_layout();
#plt.savefig('out/default-data.png');